In [26]:
#in: picke files with X_train, y_train, X_test, y_test 
#out: picke files with features  for test and train (size_diff_train, partial_ratio_test etc) 
import numpy as np
import pickle
import pandas as pd
!pip install fuzzywuzzy
!pip install python-Levenshtein
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from tqdm import tqdm
import gensim
import nltk
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis


In [13]:
from google.colab import drive
drive.mount('/content/gdrive')
path_to_objects = '/content/gdrive/My Drive/UCU-2019-final-project-storage/serialization_objects/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:

X_train = pickle.load(open(path_to_objects + 'X_train.p', 'rb'))
y_train = pickle.load(open(path_to_objects + 'y_train.p', 'rb'))
X_test = pickle.load(open(path_to_objects + 'X_test.p', 'rb'))
y_test = pickle.load(open(path_to_objects + 'y_test.p', 'rb'))


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords

stops = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
def add_feature(dataset, feature_name, feature):
  dataset[feature_name] = dataset.apply(feature, axis=1)
 

In [0]:
len_diff = lambda data: abs(len(str(data['question1'])) - len(str(data['question2'])))

token_sort_ratio = lambda data: fuzz.token_sort_ratio(str(data['question1']), str(data['question2']))

token_set_ratio = lambda data: fuzz.token_set_ratio(str(data['question1']), str(data['question2']))

partial_ratio = lambda data: fuzz.partial_ratio(str(data['question1']), str(data['question2']))

def intersection_ratio(data):
  q1 = {}
  q2 = {}
  for word in str(data['question1']).lower().split():
    if word not in stops:
      q1[word] = q1.get(word, 0) + 1
  for word in str(data['question2']).lower().split():
    if word not in stops:
      q2[word] = q2.get(word, 0) + 1
  q1_shared_count = sum([q1[w] for w in q1 if w in q2])
  q2_shared_count = sum([q2[w] for w in q2 if w in q1])
  total = sum(q1.values()) + sum(q2.values())
  return (q1_shared_count + q2_shared_count) / total



token_ratio = lambda data: fuzz.ratio(str(data['question1']), str(data['question2']))

jaccard_distance = lambda data: nltk.jaccard_distance(set(str(data['question1'])), set(str(data['question2'])))
n_question_marks_diff = lambda data: abs(str(data['question1']).count('?') - str(data['question2']).count('?'))
n_capital_letters_diff = lambda data: abs(sum(map(str.isupper, str(data['question1']))) - sum(map(str.isupper, str(data['question2']))))




In [16]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2019-05-01 10:03:58--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.184.245
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.184.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  32.0MB/s    in 47s     

2019-05-01 10:04:46 (33.1 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [0]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [0]:
def sent2vec(s):
    words = str(s).lower()
    words = nltk.word_tokenize(words)
    words = [w for w in words if not w in stops]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v / np.sqrt((v ** 2).sum())

In [0]:
def createVectors(data):
  q1_vecs = np.zeros((data.shape[0], 300))
  error_count = 0

  for i, q in tqdm(enumerate(data.question1.values)):
      q1_vecs[i, :] = sent2vec(q)

  q2_vecs  = np.zeros((data.shape[0], 300))
  for i, q in tqdm(enumerate(data.question2.values)):
      q2_vecs[i, :] = sent2vec(q)
      
  return q1_vecs, q2_vecs

In [37]:
q1_vecs_train, q2_vecs_train = createVectors(X_train)

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
270872it [00:55, 4895.41it/s]
270872it [00:56, 4773.84it/s]


In [38]:
q1_vecs_test, q2_vecs_test = createVectors(X_test)

0it [00:00, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
133415it [00:26, 5043.56it/s]
133415it [00:29, 4599.75it/s]


In [40]:
def add_distances(data, q1v, q2v):
  data['cosine_distance'] = [cosine(x, y) for (x, y) in zip(np.nan_to_num(q1v),
                                                            np.nan_to_num(q2v))]

  data['cityblock_distance'] = [cityblock(x, y) for (x, y) in zip(np.nan_to_num(q1v),
                                                            np.nan_to_num(q2v))]

  data['jaccard_distance'] = [jaccard(x, y) for (x, y) in zip(np.nan_to_num(q1v),
                                                            np.nan_to_num(q2v))]

  data['canberra_distance'] = [canberra(x, y) for (x, y) in zip(np.nan_to_num(q1v),
                                                            np.nan_to_num(q2v))]

  data['euclidean_distance'] = [euclidean(x, y) for (x, y) in zip(np.nan_to_num(q1v),
                                                            np.nan_to_num(q2v))]

  data['minkowski_distance'] = [minkowski(x, y, 3) for (x, y) in zip(np.nan_to_num(q1v),                                                           
                                                            np.nan_to_num(q2v))]
  
  data['braycurtis_distance'] = [braycurtis(x, y) for (x, y) in zip(np.nan_to_num(q1v),
                                                            np.nan_to_num(q2v))]
  
add_distances(X_train, q1_vecs_train, q2_vecs_train)




/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:702: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:1160: RuntimeWarning: invalid value encountered in double_scalars
  return l1_diff.sum() / l1_sum.sum()


In [41]:
add_distances(X_test, q1_vecs_test, q2_vecs_test)

/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:702: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:1160: RuntimeWarning: invalid value encountered in double_scalars
  return l1_diff.sum() / l1_sum.sum()


In [0]:
del model, q1_vecs_train, q2_vecs_train, q1_vecs_test, q2_vecs_test

In [0]:
add_feature(X_train, "len_diff", len_diff)
add_feature(X_train, "token_sort_ratio", token_sort_ratio)
add_feature(X_train, 'token_ratio', token_ratio)
add_feature(X_train, "intersection_ratio", intersection_ratio)
add_feature(X_train, "token_set_ratio", token_set_ratio)
add_feature(X_train, "partial_ratio", partial_ratio)
add_feature(X_train, 'jaccard_distance', jaccard_distance)
add_feature(X_train, "n_capital_letters_diff", n_capital_letters_diff)
add_feature(X_train, 'n_question_marks_diff', n_question_marks_diff)

In [0]:
add_feature(X_test, "len_diff", len_diff)
add_feature(X_test, "token_sort_ratio", token_sort_ratio)
add_feature(X_test, 'token_ratio', token_ratio)
add_feature(X_test, "intersection_ratio", intersection_ratio)
add_feature(X_test, "token_set_ratio", token_set_ratio)
add_feature(X_test, "partial_ratio", partial_ratio)
add_feature(X_test, 'jaccard_distance', jaccard_distance)
add_feature(X_test, "n_capital_letters_diff", n_capital_letters_diff)
add_feature(X_test, 'n_question_marks_diff', n_question_marks_diff)

In [44]:
X_test.head(10)

,qid1,qid2,question1,question2,token_sort_ratio,token_ratio,intersection_ratio,token_set_ratio,partial_ratio,jaccard_distance,n_capital_letters_diff,n_question_marks_diff,cosine_distance,cityblock_distance,canberra_distance,euclidean_distance,minkowski_distance,braycurtis_distance
id,,,,,,,,,,,,,,,,,,
92217,154416,154417,Thoughts on the new Pakistani blockbuster movi...,What do you think of the new Pakistani movie W...,69,64,0.727273,73,64,1.0,3,0,0.165341,7.769248,127.172923,0.575050,0.261045,0.300964
191143,12400,168520,How do I come out of comfort zone?,How exactly can I get out of this comfort zone...,56,57,0.222222,86,71,1.0,0,0,0.191389,8.635870,134.466022,0.618691,0.277733,0.325870
22640,42469,42470,What are the best ways to fake your own death?,What are the worst ways to fake one's own death?,83,87,0.666667,89,85,1.0,0,0,0.096990,5.966336,102.987159,0.440431,0.200047,0.216731
348814,1178,52837,What is the meaning of life? Whats our purpose...,What's the purpose of a human life?,61,51,0.444444,83,57,1.0,2,1,0.284874,10.532384,149.797132,0.754816,0.338221,0.406219
259163,374897,374898,How do I better communicate with my boyfriend?,How can I better communicate with my boyfriend...,65,65,0.444444,97,93,1.0,0,0,0.126197,6.988566,113.841849,0.502387,0.224956,0.257012
198471,213362,290257,How much weight can I lose in a month if I cyc...,Is it safe to run everyday to lose weight?,47,41,0.285714,69,50,1.0,2,0,0.262685,10.098605,146.736135,0.724824,0.325255,0.389079
83766,16814,23070,Can we time travel anyhow?,Is time travel to 2010 possible?,46,52,0.571429,61,54,1.0,0,0,0.188101,8.353378,127.730048,0.613353,0.281684,0.316801
98058,163015,163016,What are colors that start with the letter k?,What are colors that start with the letter n?,95,98,0.750000,98,98,1.0,0,0,0.209341,9.065644,138.520391,0.647057,0.289407,0.347238
110810,32893,107140,Which test would be easier: IELTS or PTE (Pear...,Is the PTE or IELTS easy?,46,37,0.363636,67,48,1.0,4,0,0.561829,14.650415,182.467787,1.060027,0.482449,0.617053


In [0]:
pickle.dump(X_train, open(path_to_objects+"X_train_statistic.p", 'wb'))
pickle.dump(X_test, open(path_to_objects+"X_test_statistic.p", 'wb'))